- [Training](#training)
- [Testing (needs to be run after training)](#testing)

# Training

In [1]:
from bag_of_words import *

In [2]:
is_working_with_easy_dataset = True

In [3]:
if is_working_with_easy_dataset:
    input_train_filename = "jd_easy_train.csv"
    input_test_filename = "jd_easy_test.csv"
else:
    input_train_filename = "jd_difficult_train.csv"
    input_test_filename = "jd_difficult_test.csv"

jd_train = pd.read_csv(input_train_filename, keep_default_na=False)
jd_test = pd.read_csv(input_test_filename, keep_default_na=False)

In [4]:
jd_train["tokens"] = jd_train.description.map(
    lambda x: get_text(x)).map(
    lambda x: get_tokenized_text(x))

In [5]:
jd_train["is_noun_or_adj"] = [get_nouns_and_adjs(text) for text in jd_train.tokens]

In [7]:
jd_train["lemmatized_tokens"] = jd_train.tokens.map(
    lambda x: [lemmatize(word) for word in x])

In [9]:
jd_train.lemmatized_tokens

0       [Roles, &, Responsibilities, Responsibilities,...
1       [LECTURER, –, GAME, ART, –, SINGAPORE, CAMPUS,...
2       [HR, PROJECT, SPECIALIST, Our, team, in, Singa...
3       [Job, Description, Our, client, is, in, the, i...
4       [Responsibilities, Design, EDM, ,, email, camp...
                              ...                        
3177    [Job, Description, Job, Description, :, Postin...
3178    [Sales, Executives, Date, :, 03-Mar-2023, Loca...
3179    [Company, Tricove, Tech, Pte, Ltd, digitalcard...
3180    [Account, Manager, (, HR, Software, Sales, ), ...
3181    [Working, Days, :, Mon, to, Fri, ,, 10AM, to, ...
Name: lemmatized_tokens, Length: 3182, dtype: object

In [7]:
jd_train.head()

,category,description,tokens,is_noun_or_adj,lemmatized_tokens
0,software+engineer,"<div class=""jobsearch-jobDescriptionText"" dir=...","[Roles, &, Responsibilities, Responsibilities,...","[True, False, True, True, False, True, False, ...","[Roles, &, Responsibilities, Responsibilities,..."
1,arts,"<div class=""jobsearch-jobDescriptionText"" dir=...","[LECTURER, –, GAME, ART, –, SINGAPORE, CAMPUS,...","[True, True, True, True, True, True, True, Fal...","[LECTURER, –, GAME, ART, –, SINGAPORE, CAMPUS,..."
2,hr,"<div class=""jobsearch-jobDescriptionText"" dir=...","[HR, PROJECT, SPECIALIST, Our, team, in, Singa...","[True, True, True, False, True, False, True, F...","[HR, PROJECT, SPECIALIST, Our, team, in, Singa..."
3,arts,"<div class=""jobsearch-jobDescriptionText"" dir=...","[Job, Description, Our, client, is, in, the, i...","[True, True, False, True, False, False, False,...","[Job, Description, Our, client, is, in, the, i..."
4,arts,"<div class=""jobsearch-jobDescriptionText"" dir=...","[Responsibilities, Design, EDM, ,, email, camp...","[True, True, True, False, True, True, False, T...","[Responsibilities, Design, EDM, ,, email, camp..."


In [10]:
top_stems = get_top_lemmatized_noun_adj(jd_train, 50)
len(top_stems)

116

In [16]:
pd.DataFrame(top_stems).to_csv("top_stems.csv", index=False, header=False)

In [11]:
description_top_stem_vector_train = pd.DataFrame(get_tfidf(jd_train.lemmatized_tokens, top_stems), columns=top_stems)
bag_train = pd.concat([jd_train.category, description_top_stem_vector_train], axis=1)
bag_train.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


,category,good,visual,platform,development,technical,tool,material,Location,candidate,...,change,Good,system,asset,Sales,game,Requirements,knowledge,client,communication
0,software+engineer,0.000000,0.000000,0.0,0.282396,0.281375,0.000000,0.0,0.0,0.000000,...,0.171654,0.0,0.370467,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1,arts,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.821889,0.0,0.092776,0.000000,0.000000
2,hr,0.076998,0.000000,0.0,0.000000,0.000000,0.109376,0.0,0.0,0.000000,...,0.251116,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.069475
3,arts,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.072032,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.071156,0.082946,0.268595
4,arts,0.097715,0.252972,0.0,0.000000,0.000000,0.069402,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.088168


# Testing

In [ ]:
top_stems = [x[0] for x in pd.read_csv("top_stems.csv", header=None).values.tolist()]

In [10]:
jd_test["tokens"] = jd_test.description.map(
    lambda x: get_text(x)).map(
    lambda x: get_tokenized_text(x))

jd_test["is_noun_or_adj"] = [get_nouns_and_adjs(text) for text in jd_test.tokens]

jd_test["lemmatized_tokens"] = jd_test.tokens.map(
    lambda x: [lemmatize(word) for word in x])

description_top_stem_vector_test = pd.DataFrame(get_tfidf(jd_test.lemmatized_tokens, top_stems), columns=top_stems)
bag_test = pd.concat([jd_test.category, description_top_stem_vector_test], axis=1)
bag_test.head()

/Users/wangjiayi/Desktop/job-search-nlp/bag_of_words.py:10: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 10 of the file /Users/wangjiayi/Desktop/job-search-nlp/bag_of_words.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html_text)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


,category,technical,medium,ability,HR,Software,Adobe,Requirements,environment,understanding,...,game,Responsibilities,resume,Photoshop,target,service,asset,digital,employee,office
0,software+engineer,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.215847,0.124923,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,sales,0.0,0.000000,0.180794,0.0,0.0,0.0,0.0,0.000000,0.107779,...,0.0,0.0,0.000000,0.0,0.145563,0.000000,0.0,0.000000,0.000000,0.000000
2,sales,0.0,0.000000,0.049153,0.0,0.0,0.0,0.0,0.000000,0.058604,...,0.0,0.0,0.000000,0.0,0.000000,0.049153,0.0,0.000000,0.064307,0.063536
3,sales,0.0,0.251614,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,arts,0.0,0.000000,0.116951,0.0,0.0,0.0,0.0,0.040155,0.000000,...,0.0,0.0,0.051128,0.0,0.000000,0.000000,0.0,0.050034,0.000000,0.000000


In [11]:
if is_working_with_easy_dataset:
    output_train_filename = "bag_easy_train.csv"
    output_test_filename = "bag_easy_test.csv"
else:
    output_train_filename = "bag_difficult_train.csv"
    output_test_filename = "bag_difficult_test.csv"

bag_train.to_csv(output_train_filename, index=False)
bag_test.to_csv(output_test_filename, index=False)